## Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
import requests
import json
import traceback
from decimal import Decimal
from abc import ABC, abstractmethod

In [2]:
def get_web_data(url:str)->tuple:
    """Функция позволяет получить веб-данные по указанному url.
    
    Аргументы функции:
    url -- строка в формате http
    
    Возвращаемое значение: (код завершения функции, декодированный веб-контент или None)
    """
    
    if not isinstance(url, str):
        raise ValueError('Переменная url должна иметь тип str')
    
    # Получаем данные из API, таймаут 10 секунд
    try:
        response = requests.get(url, timeout=10)
    except:
        extend_error = traceback.format_exc()
        print(f'Не удалось получить данные из сервиса {url}\n{extend_error}')
        return 1, None
    
    # Если веб-сервер возвращает код отличный от 200, значит произошла ошибка, уведомляем пользователя, возвращаем None
    if response.status_code != 200:
        print(f'Не удалось получить данные из сервиса {url}, сервер вернул ошибку {response.status_code} {response.reason}')
        return 2, None
 
    return 0, response.content.decode(response.encoding)
    
    
def get_exhange_rates()->dict:
    """Функция получает курсы валют из сервиса https://www.cbr-xml-daily.ru/daily_json.js
    
    Возвращаемое значение: словарь курсов валют или пустой словарь
    
    Формат возвращаемого словаря:
    {"КОД ВАЛЮТЫ": {
                    "ID": ЗНАЧЕНИЕ ИДЕНТИФИКАТОРА ВАЛЮТЫ тип str,
                    "NumCode": ЗНАЧЕНИЕ КОДА ВАЛЮТЫ тип str,
                    "CharCode": СИМВОЛЬНОЕ ОБОЗНАЧЕНИЕ ВАЛЮТЫ тип str,
                    "Nominal":  НОМИНАЛ ВАЛЮТЫ тип int,
                    "Name": НАЗВАНИЕ ВАЛЮТЫ  тип str,
                    "Value": ТЕКУЩЕЕ ЗНАЧЕНИЕ КУРСА ВАЛЮТЫ  тип float,
                    "Previous": ПРЕДЫДУЩЕЕ ЗНАЧЕНИЕ КУРСА ВАЛЮТЫ  тип float
                    }
    }
    """
    
    # Ссылка на сервис для получения курсов валют
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'
    
    # Поулчаем даныне из сервиса, в случае ошибки получения данных уведомляем пользователя и возвращаем пустой словарь
    return_code, web_content = get_web_data(url)
    if return_code != 0:
        print('Не удалось сформировать словарь курсов валют из-за ошибки получения данных из API')
        return {}
 
    # Формируем итоговый словарь функции, в случае ошибки во время операции уведомляем пользователя 
    # и возвращаем пустой словарь
    try:
        exhange_rates = json.loads(web_content)['Valute']
    except:
        extend_error = traceback.format_exc()
        print(f'Не удалось сформировать словарь валют после получения данных из API\n{extend_error}')
        return {}
        
    return exhange_rates


def name_currency_with_maximum_value(exhange_rates:dict)->str:
    """
    Функция для поиска валюты с максималным значением курса.
    
    Аргументы функции:
    exhange_rates -- словарь курсов валют. 
    Словарь Должен иметь формат:
        {"КОД ВАЛЮТЫ": {
                    "Name": НАЗВАНИЕ ВАЛЮТЫ  тип str,
                    "Value": ТЕКУЩЕЕ ЗНАЧЕНИЕ КУРСА ВАЛЮТЫ  тип float
                    }
          ...
        }
        
    Возвращаемое значение: полное наименование валюты или None
    """
    
    if not isinstance(exhange_rates, dict):
        raise ValueError('Переменная exhange_rates должна иметь тип dict')
    
    # Сортируем словарь по значению текущего курса от большего к меньшему
    try:
        sorted_exhange_rates = sorted(exhange_rates.values(), key=lambda x: x['Value'], reverse=True)
    except:
        extend_error = traceback.format_exc()
        print(f'Произошла ошибка при сортировке словаря exhange_rates\n{extend_error}')
        return None
    
    return sorted_exhange_rates[0]['Name']
    
def main():    
    exhange_rates = get_exhange_rates()
    if exhange_rates:
        print(name_currency_with_maximum_value(exhange_rates))
    else:
        print('Название валюты с максимальным курсом не найдено')
        
main()

СДР (специальные права заимствования)


## Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [3]:
class Rate:
    def __init__(self, format_:str='value', diff:bool=False):
        
        self.__check_format_param(format_)
        self.__check_diff_param(diff)

        self._format = format_.lower()
        self._diff = diff
        
    @staticmethod
    def __check_format_param(format_):
        """
        Функция проверки параметра format_ при пользовательском изменении  
        """
        if format_.lower() not in ('value', 'full'):
            raise ValueError('Параметр format_ должен иметь значение "value" или "full"')
            
    @staticmethod
    def __check_diff_param(diff):
        """
        Функция проверки параметра diff при пользовательском изменении 
        """
        if not isinstance(diff, bool):
            raise ValueError('Параметр diff должен иметь тип bool')
    
    @property
    def format(self):
        return self._format
    @format.setter
    def format(self, format):
        self.__check_format_param(format)
        self._format = format
        
    @property
    def diff(self):
        return self._diff
    @diff.setter
    def diff(self, diff):
        self.__check_diff_param(diff)
        self._diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        
        # Получаем данные из API, таймаут 10 секунд. В случае ошибки возвращаем None
        url = 'https://www.cbr-xml-daily.ru/daily_json.js'
        try:
            self._r = requests.get(url, timeout=10)
        except:
            return None

        # Если веб-сервер возвращает код отличный от 200, значит произошла ошибка и возвращаем None
        if self._r.status_code != 200:
            return None
        
        return self._r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self._format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        if response is None:
            return 'Error'
        
        if currency in response:
            if self._format == 'full':
                return response[currency]
            
            if self._format == 'value':
                if self._diff:
                    return Decimal(str(response[currency]['Value'])) - Decimal(str(response[currency]['Previous']))     
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [4]:
rate_obj = Rate(format_='value', diff=True)
print('Разница курса EUR:', rate_obj.eur())
rate_obj.diff = False
print('Текущий курс EUR:', rate_obj.eur())
rate_obj.format='full'
print('Подробная инфомрация о курсе EUR:', rate_obj.eur())

Разница курса EUR: -0.0969
Текущий курс EUR: 89.0384
Подробная инфомрация о курсе EUR: {'ID': 'R01239', 'NumCode': '978', 'CharCode': 'EUR', 'Nominal': 1, 'Name': 'Евро', 'Value': 89.0384, 'Previous': 89.1353}


## Задание 3

Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

In [5]:
class EmployeeAbstract(ABC):
    
    @abstractmethod
    def grade_up(self):
        """Повышает уровень сотрудника"""
        pass
    
    @abstractmethod
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        pass
    
    @abstractmethod
    def check_if_it_is_time_for_upgrade(self):
        """Проверяется требуется ли повышение уровня"""
        pass
    

class Employee(EmployeeAbstract):
    def __init__(self, name:str, seniority:'int >= 0'):
        self.name = name
        if not isinstance(seniority, int) or seniority < 0:
            raise ValueError('Параметр seniority должен быть типом int и быть больше или равным 0')
        self.seniority = seniority
        
        self.grade = 1
        
    def __call__(self):
        return self.check_if_it_is_time_for_upgrade()
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass


class Designer(Employee):
       
    def __call__(self, prize_count=0):
        return self.check_if_it_is_time_for_upgrade(prize_count=prize_count)
    
    def check_if_it_is_time_for_upgrade(self, prize_count:'int >= 0'=0):
        # Для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        # Если дизайнер получил международную премию, то добавляем дополнительные 2 балла за каждую премию.
        # Учитываем только новые премии, которые появились с момента прошлой акредитации или были не учтены ранее.
        # Количество неучтеных международных премий на момент акредитации передаются в параметре prize_count типа int
        
        if not isinstance(prize_count, int) or prize_count < 0:
            raise ValueError('Параметр prize_count должен быть типом int и быть больше или равным 0')
        
        bonus_points = 1 + prize_count*2 if prize_count else 1
            
        while bonus_points > 0:
            self.seniority += 1
            if self.seniority % 7 == 0:
                self.grade_up()
            bonus_points -= 1
        
        # публикация результатов
        return self.publish_grade()
    
    
designer_1 = Designer('Павел', 0)
designer_1()
designer_1()
designer_1()
designer_1()
designer_1()
designer_1()
designer_1()
designer_1(prize_count=3)
designer_1(prize_count=1)
designer_1()

Павел 1
Павел 1
Павел 1
Павел 1
Павел 1
Павел 1
Павел 2
Павел 3
Павел 3
Павел 3
